# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/ID_Conditional_VAE/baseline/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_types[0]}_features.pkl')
#dataloaders_dict = modeler.make_dataloader(ext_data)

In [9]:
#for sample in dataloaders_dict['train']:
#    hoge = sample['features']

## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    net = Model(in_features, mid_size, latent_size)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, scheduler, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [11]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [12]:
for machine_type in machine_types:
    run(machine_type)

2021-04-20 21:27:55,538 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-04-20 21:27:55,539 - 00_train.py - INFO - MAKE DATA_LOADER
2021-04-20 21:27:55,573 - 00_train.py - INFO - TRAINING


use: cuda:0


2021-04-20 21:27:57,418 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:04<00:00, 44.85it/s]
2021-04-20 21:28:01,634 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:00<00:00, 144.70it/s]
2021-04-20 21:28:01,789 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 143.04it/s]


,AUC,pAUC
Source_0,0.567100,0.491053
Source_1,0.468500,0.491579
Source_2,0.523700,0.477895
Target_0,0.524100,0.510000
Target_1,0.509400,0.497895
Target_2,0.529600,0.511053
mean,0.520400,0.496579
h_mean,0.518732,0.496311


2021-04-20 21:28:03,013 - pytorch_modeler.py - INFO - epoch:1/100, train_losses:0.009775, val_AUC_hmean:0.518732, val_pAUC_hmean:0.496311, best_flag:True
2021-04-20 21:28:03,013 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:03<00:00, 49.59it/s]
2021-04-20 21:28:06,827 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:00<00:00, 138.62it/s]
2021-04-20 21:28:06,981 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 126.35it/s]


...

100%|██████████| 189/189 [00:03<00:00, 48.53it/s]
2021-04-20 22:14:47,390 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 125.76it/s]
2021-04-20 22:14:47,565 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 117.03it/s]
2021-04-20 22:14:47,757 - pytorch_modeler.py - INFO - epoch:56/100, train_losses:0.003193, val_AUC_hmean:0.579977, val_pAUC_hmean:0.511499, best_flag:False
2021-04-20 22:14:47,759 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:04<00:00, 47.06it/s]
2021-04-20 22:14:51,778 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 120.72it/s]
2021-04-20 22:14:51,960 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 114.99it/s]
2021-04-20 22:14:52,146 - pytorch_modeler.py - INFO - epoch:57/100, train_losses:0.003207, val_AUC_hmean:0.582597, val_pAUC_hmean:0.514288, best_flag:False
2021-04-20 22:14:52,147 - pytorch_modeler.py - INFO - train
100%

100%|██████████| 19/19 [00:00<00:00, 133.18it/s]
2021-04-20 22:16:30,248 - pytorch_modeler.py - INFO - epoch:72/100, train_losses:0.003143, val_AUC_hmean:0.580480, val_pAUC_hmean:0.513642, best_flag:False
2021-04-20 22:16:30,249 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:03<00:00, 49.01it/s]
2021-04-20 22:16:34,107 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 131.30it/s]
2021-04-20 22:16:34,275 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 125.19it/s]
2021-04-20 22:16:34,447 - pytorch_modeler.py - INFO - epoch:73/100, train_losses:0.003082, val_AUC_hmean:0.577706, val_pAUC_hmean:0.512068, best_flag:False
2021-04-20 22:16:34,448 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:03<00:00, 48.59it/s]
2021-04-20 22:16:38,340 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 117.12it/s]
2021-04-20 22:16:38,531 - pytorch_modeler.py - INFO - valid_target
100%

100%|██████████| 189/189 [00:06<00:00, 28.53it/s]
2021-04-20 22:18:13,467 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 114.76it/s]
2021-04-20 22:18:13,658 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 104.46it/s]
2021-04-20 22:18:13,859 - pytorch_modeler.py - INFO - epoch:89/100, train_losses:0.003095, val_AUC_hmean:0.581487, val_pAUC_hmean:0.514918, best_flag:False
2021-04-20 22:18:13,860 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [00:05<00:00, 31.50it/s]
2021-04-20 22:18:19,862 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 20/20 [00:00<00:00, 120.94it/s]
2021-04-20 22:18:20,042 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:00<00:00, 108.29it/s]
2021-04-20 22:18:20,238 - pytorch_modeler.py - INFO - epoch:90/100, train_losses:0.003094, val_AUC_hmean:0.579509, val_pAUC_hmean:0.512180, best_flag:False
2021-04-20 22:18:20,239 - pytorch_modeler.py - INFO - train
100%

,AUC,pAUC
Source_0,0.570900,0.501053
Source_1,0.645900,0.538421
Source_2,0.594652,0.527629
Target_0,0.556900,0.534211
Target_1,0.577755,0.533620
Target_2,0.539696,0.486938
mean,0.580967,0.520312
h_mean,0.579101,0.519574


elapsed time: 501.287478924 [sec]
